# Extract rock type

This notebook extracts the fractional makeup of rock types (igneous, sedimentary, metamorphic) present in the catchment area of each debris flow location.

Potential improvement: restrict to area with a slope of 23$^\circ$ or more.  This needs a new catchment area.

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

Load a few columns from the sites file which contains the catchment geometry (computed using `extract_contributing_region.ipynb`).
Only keep one row per site, and set `SiteID` column as index:

In [ ]:
#pqfile= "staley16_observations_catchment_fuelpars_v3.parquet"
pgfile ="staley16_sites_catchment_fuelpars_v3.parquet"
modelIF=gpd.read_parquet(pgfile).set_crs('epsg:4326')

In [ ]:
modelIF=modelIF[["newarea","geometry","snapdist"]]
display(modelIF.shape)
modelIF.head(n=2)

Geological map needs to be retrieved for every state.  First get a list of states, then download and unpack the map for each state:

In [ ]:
states=gpd.read_parquet("staley16_debrisflow.parquet")["state"].drop_duplicates().to_list()
states

In [ ]:
odir = "geological_map"
from os.path import isfile, isdir
from posix import mkdir 

if not isdir(odir):
    mkdir(odir)

for state in states:
    url="https://mrdata.usgs.gov/geology/state/shp/" + state + ".zip"
    ofile=odir + "/" + state + ".zip"
    print(url)
    if not isfile(ofile):
        !curl -o $ofile $url 
        !unzip -d $odir $ofile

Load the geological map of each state and concatenate into a single GeoPandas dataframe:

In [ ]:
geols=[gpd.read_file(odir + "/" + state + "_geol_poly.shp") for state in states]

In [ ]:
geol=gpd.GeoDataFrame(pd.concat(geols, ignore_index=True), crs='epsg:4326')

In [ ]:
geol["GENERALIZE"].unique()

Simplify geology labels, keeping only part before comma:

In [ ]:
def simplify_geol(gen):
    #if gen.lower().find("and") >= 0:
    if gen == "Water":
        return gen
    else:
        return gen.split(",")[0]
    
geol["rocktype"] = geol["GENERALIZE"].apply(simplify_geol)

In [ ]:
geol["rocktype"].unique()

In [ ]:
print(geol["geometry"].iat[0])

Carry out a spatial join between geological map and catchment areas. The resulting dataframe has one row for every match between catchment area and geological units; sometimes more than one per catchment.

In [ ]:
mrg=modelIF.sjoin(geol, how="left", predicate='intersects')
mrg.head(n=2)

In [ ]:
mrg.loc[9,:]

In [ ]:
geol.loc[7533,:]

In [ ]:
import plotly.express as px
import plotly
plotly.io.templates.default = 'plotly_dark'

Plotting up one of the catchment aras (for Site 9) and the geological units that make it up:

In [ ]:
idxc=[9]

mif=modelIF.loc[idxc,:].set_crs('epsg:4326')
mif["Site"] = mif.index.astype(str)

center={"lon": mif["geometry"].iat[0].centroid.x,
        "lat": mif["geometry"].iat[0].centroid.y}

fig1=px.choropleth_mapbox(mif,mif["geometry"],
                     locations=idxc,
                     zoom=12,
                     center=center,
                     color="Site",
                     color_discrete_map={'9': 'green'},
                     height=500,
                     opacity=0.5,
                     mapbox_style="stamen-terrain")

idx = mrg.loc[idxc,"index_right"].to_list()
gidx=geol.loc[idx,:].to_crs('epsg:4326')
fig2=px.choropleth_mapbox(gidx, gidx["geometry"],
                     locations=idx,
                     color=gidx["GENERALIZE"],
                     opacity=0.5)
                          
fig1.add_traces(fig2.data)

fig1

Counting the number of units present within each catchment area:

In [ ]:
rtypes_by_site=mrg.groupby("SiteID").agg({"rocktype": "count"})
rtypes_by_site.head()

In [ ]:
rtypes_by_site.hist(bins=np.arange(0.5,15.5,1))

Most sites have just one rock type, very few more than 5.

The function below computes the percentage of overlap of the catchment with each geological unit.

In [ ]:
catch=modelIF.loc[9,"geometry"]
unit=geol.loc[7533,"geometry"]

In [ ]:
#return amount that polygon p2 occupies of the area of polygon p1 as fraction
def getpcentoverlap(p1, p2):
    return p1.intersection(p2).area/p1.area

getpcentoverlap(catch, unit)

The spatial join returns the geometry of the left dataframe, not the overlapping area.  
Re-joining with geology dataframe to have both catchment and geological unit in the same dataframe, and applying function to get percentual contribution of each geological unit to catchment:

In [ ]:
mrg2=mrg.reset_index().merge(geol["geometry"], left_on="index_right", right_index=True, 
               suffixes=["_c","_g"], how="inner")
print(len(mrg2))

In [ ]:
mrg2["Frac"]=mrg2[["geometry_c","geometry_g"]].apply(lambda x: getpcentoverlap(x[0], x[1]), axis=1)

Querying result for Site 9:

In [ ]:
mrg2.query("SiteID == 9")[["rocktype","Frac"]]

In [ ]:
len(mrg), len(mrg2)

Because similar rock types (e.g., 'metamorphic, sedimentary' and 'metamorphic, igneous') were merged into a simple (e.g., 'metamorphic') rock type, there are duplicate entries by site and rock type in the `mrg2` dataframe.
These need to be aggregated such that only one SiteID, rocktype pair exists in the table.  Saving the result to `rtfrag`:

In [ ]:
rtfrac=mrg2[["SiteID","rocktype","Frac"]].groupby(["SiteID","rocktype"]).agg({"Frac": sum})
rtfrac.head(n=12).tail()

Now unstack to create one column per rock type:

In [ ]:
rttable=rtfrac.unstack(level=1).fillna(value=0)
rttable.columns=[a[1] for a in rttable.columns]
rttable.head()

Find sites with a water coverage fraction > 0:

In [ ]:
rttable.query("Water > 0")

Just one of the sites has a bit of water inside the catchment.  So that column can be dropped.

In [ ]:
rttable.sum(axis=0) / (len(rttable))

Ignous, metamorphic and sedimentary make up the bulk of the rock types.  The mixed categories are less than one percent each.  Analyzing in more detail:

In [ ]:
rttable[rttable["Igneous and Metamorphic"] > 0]

In [ ]:
rttable[rttable["Igneous and Sedimentary"] > 0]

In [ ]:
rttable[rttable["Metamorphic and Sedimentary"] > 0]

In [ ]:
rttable[rttable["Unconsolidated"] > 20]

Almost all sites which cover mixed category geological units also cover units of the two respective categories.  So the mixed categories can be distributed (equally) among these two types, reducing the number of columns to just 3.

In [ ]:
#rttable.head(n=1).apply(lambda x: print(type(x)), axis=1)

In [ ]:
def assign_mixed_rock_types(ts):
    mixcat=["Igneous and Metamorphic", "Igneous and Sedimentary", "Metamorphic and Sedimentary"]
    
    os=ts.copy()
    
    for mc in mixcat:
        cat1=mc.split(" and ")[0]
        cat2=mc.split(" and ")[1]
        
        # split rock types in mixed category equally among these rock types
        os[cat1] += os[mc] / 2.
        os[cat2] += os[mc] / 2.
        
        os[mc] = 0.
        
    # add water parts to sedimentary
    os["Sedimentary"] +=  os["Water"]
    os["Water"] = 0.
    
    # add and unconsolidated parts to sedimentary
    # os["Sedimentary"] += os["Unconsolidated"]
    # os["Unconsolidated"] = 0.
    
    return os

Testing the function on 5 rows:

In [ ]:
rttable.loc[336:340,:]

In [ ]:
rttable.loc[336:340,:].apply(assign_mixed_rock_types, axis=1)

Applying to full table and dropping mixed columns (which are now zero) and water:

In [ ]:
rttable4=rttable.apply(assign_mixed_rock_types, axis=1).drop(columns=["Igneous and Metamorphic",
                                                                      "Igneous and Sedimentary",
                                                                      "Metamorphic and Sedimentary",
                                                                      "Water"])

rttable4.head()

Add new column with dominant rock type:

In [ ]:
rttable4["domrt"]=rttable4.columns[rttable4.values.argmax(axis=1)]

Read data file that includes observations (debris flow, no debris flow, multiple per site):

In [ ]:
modelIFo=gpd.read_parquet("staley16_observations_catchment_fuelpars_v3.parquet").\
    drop(columns=['NB', 'GR', 'GS', 'SH', 'TU', 'TL', 'dom', 'newarea','snapdist','geometry'])
modelIFo.shape

Now join with rock type table on `SiteID`:

In [ ]:
mrgdata=modelIFo.merge(rttable4, left_on="SiteID", right_index=True)#.merge(siteinfos, left_on="SiteID", right_index=True)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')

Create plot to rock type distribution for debris flow / no debris flow events:

In [ ]:
fig,ax=plt.subplots(figsize=(12,10), ncols=2, nrows=2)

nodebfl=mrgdata.query("response == 0")
debfl=mrgdata.query("response == 1")

rts=rttable4.columns[:-1]

usedens=True

for n,rtype in enumerate(rts):

    ax.flatten()[n].hist(nodebfl[rtype], histtype="bar", color="gray", density=usedens, label="No debris flow")
    ax.flatten()[n].hist(debfl[rtype], histtype="step", color="red", density=usedens, label="debris flow")
    if n==0:
        ax.flatten()[n].legend(loc="upper center")
    ax.flatten()[n].set_title(rtype)

Calculating debris flow likelihood by dominant rock type:

In [ ]:
nodf_catcont=mrgdata.query("response == 0").groupby("domrt")["domrt"].count()
df_catcont=mrgdata.query("response == 1").groupby("domrt")["domrt"].count()

In [ ]:
catcont=pd.DataFrame({"No Debris Flow": nodf_catcont, "Debris Flow": df_catcont})
catcont

In [ ]:
catcont / catcont.sum(axis=0)

In [ ]:
catcont["DFProb"] = catcont["Debris Flow"]/(catcont["No Debris Flow"] + catcont["Debris Flow"])
catcont

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [ ]:
fig2,ax2=plt.subplots(figsize=(5,6))
fig2.subplots_adjust(bottom=0.2)
catcont[["No Debris Flow","Debris Flow"]].plot(kind="bar", ax=ax2)
fig2.savefig("rocktype_frequency.png", dpi=300)

In [ ]:
mrgdata.to_parquet("staley16_observations_catchment_fuelpars_rocktype_v3.parquet")

Creating a dataframe which contains both the catchment and dominant rock type, along site coordinates:

In [ ]:
sitelocs=modelIFo[["lon","lat","SiteID","contributingarea_km2"]].drop_duplicates().set_index("SiteID", drop=True)

In [ ]:
plottab=modelIF.join(sitelocs).join(rttable4).to_crs('epsg:4326')

Rename column `domrt`:

In [ ]:
cols=plottab.columns.to_list()
cols.remove("domrt")
cols.append("Dominant Rock Type")
plottab.columns=cols

In [ ]:
#how to simplify a shapely geometry
#plottab.loc[700,"geometry"].simplify(0.00025)

In [ ]:
tol=0.00025
plottab["geometry_simple"]=plottab["geometry"].map(lambda x: x.simplify(tol))

In [ ]:
center={"lon": plottab["geometry"].centroid.x.median(),
        "lat": plottab["geometry"].centroid.y.median()}

In [ ]:
plottab["Legend"] = "DF Site"
plottab["size"] = 10

In [ ]:
fig1=px.choropleth_mapbox(plottab, plottab["geometry_simple"],
                     locations=plottab.index,
                     color="Dominant Rock Type",
                     #color="snapdist",
                     hover_data=["Igneous","Metamorphic","Sedimentary","Unconsolidated"],
                     center=center,
                     mapbox_style="stamen-terrain",
                     height=800,
                     zoom=4,
                     opacity=0.5)

fig2=px.scatter_mapbox(plottab.reset_index(), lon="lon", lat="lat", color="Legend", 
                      color_discrete_map={"DF Site": "orange"},
                      size="size", size_max=5,
                      hover_data=["SiteID", "contributingarea_km2", "newarea","snapdist"])

fig1.add_traces(fig2.data)

fig1

In [ ]:
fig1.write_html("staley16_rock_type_mod.html")